In [1]:
import matplotlib.pyplot as plt

# Leer metadatos de archivos .tif

Estas celdas definen funciones para listar archivos `.tif` en una ruta y extraer sus medidas (width, height, bandas, dtype, CRS, resolución y bounds).

Si `rasterio` no está instalado, se usa `gdalinfo -json` como fallback (requiere GDAL instalado).

In [2]:
# Funciones para encontrar y leer metadatos de .tif
import os
import glob
import json
import subprocess
from pathlib import Path
try:
    import rasterio
    _HAS_RASTERIO = True
except Exception:
    _HAS_RASTERIO = False

def list_tifs(path):
    """Devuelve una lista con rutas absolutas a archivos .tif/.tiff dentro de `path` (recursivo)."""
    p = Path(path)
    if not p.exists():
        return []
    patterns = ['**/*.tif', '**/*.tiff']
    files = []
    for pat in patterns:
        files.extend([str(x) for x in p.glob(pat)])
    files = sorted(list(set(files)))
    return files

def _info_rasterio(path):
    with rasterio.open(path) as src:
        info = {
            'path': path,
            'width': int(src.width),
            'height': int(src.height),
            'bands': int(src.count),
            'dtypes': src.dtypes,
            'crs': str(src.crs) if src.crs else None,
            'transform': tuple(src.transform) if hasattr(src, 'transform') else None,
            'resolution': getattr(src, 'res', None),
            'bounds': tuple(src.bounds) if hasattr(src, 'bounds') else None,
            'driver': src.driver if hasattr(src, 'driver') else None,
            'nodata': src.nodata if hasattr(src, 'nodata') else None,
        }
    return info

def _info_gdalinfo(path):
    # Fallback: usar gdalinfo -json y parsear
    try:
        out = subprocess.check_output(['gdalinfo', '-json', path], stderr=subprocess.DEVNULL)
        obj = json.loads(out)
    except Exception as e:
        return {'path': path, 'error': str(e)}
    info = {'path': path}
    size = obj.get('size') or [obj.get('size_x'), obj.get('size_y')]
    try:
        info['width'] = int(size[0])
        info['height'] = int(size[1])
    except Exception:
        info['width'] = None
        info['height'] = None
    info['bands'] = len(obj.get('bands', []))
    info['dtype'] = obj.get('bands', [{}])[0].get('dataType') if obj.get('bands') else None
    # crs: intento leer wkt o proj4
    crs = obj.get('coordinateSystem') or obj.get('crs')
    info['crs'] = crs.get('wkt') if isinstance(crs, dict) and 'wkt' in crs else str(crs)
    gt = obj.get('geoTransform')
    if gt:
        info['resolution'] = (gt[1], abs(gt[5]))
    else:
        info['resolution'] = None
    # bounds
    if 'cornerCoordinates' in obj:
        info['bounds'] = obj.get('cornerCoordinates')
    elif 'extent' in obj:
        info['bounds'] = obj.get('extent')
    else:
        info['bounds'] = None
    return info

def get_raster_info(path):
    """Devuelve un diccionario con metadatos del raster en `path`."""
    if _HAS_RASTERIO:
        try:
            return _info_rasterio(path)
        except Exception as e:
            return {'path': path, 'error': f'rasterio error: {e}'}
    else:
        # intentar gdalinfo
        return _info_gdalinfo(path)

def summarize_rasters(paths):
    """Dado un iterable de rutas, devuelve una lista de diccionarios con la información."""
    out = []
    for p in paths:
        out.append(get_raster_info(p))
    return out

def table_from_dir(path):
    """Busca .tif en `path` y muestra un resumen imprimible."""
    files = list_tifs(path)
    if not files:
        print('No se encontraron .tif en', path)
        return []
    infos = summarize_rasters(files)
    # formato simple de impresión
    for info in infos:
        print('\n' + '-'*60)
        print('Archivo:', info.get('path'))
        if 'error' in info:
            print('  ERROR:', info['error'])
            continue
        print('  width x height:', info.get('width'), 'x', info.get('height'))
        print('  bands:', info.get('bands'))
        print('  dtype(s):', info.get('dtypes') or info.get('dtype'))
        print('  crs:', info.get('crs'))
        print('  resolution:', info.get('resolution'))
        print('  bounds:', info.get('bounds'))
    return infos

In [10]:
# Ejemplo de uso sobre las rutas que mencionaste
dir_mnc = r"/Users/joanespada/Desktop/ADE/Recorte&Visualizacion/ade_modelo_agricola/INTA-MNC"
dir_ndvi = r"/Users/joanespada/Desktop/ADE/Recorte&Visualizacion/ade_modelo_agricola/Rasters-NDVI"

print('>> Revisando INTA-MNC:')
infos_mnc = table_from_dir(dir_mnc)

print('\n>> Revisando Rasters-NDVI:')
infos_ndvi = table_from_dir(dir_ndvi)

# Para devolver los resultados en una variable dentro del notebook:
results = {'INTA-MNC': infos_mnc, 'Rasters-NDVI': infos_ndvi}

# Si quieres verlos como tabla (pandas) instancia un DataFrame opcionalmente:
try:
    import pandas as pd
    rows = []
    for k,v in results.items():
        for info in v:
            r = info.copy()
            r['source_dir'] = k
            rows.append(r)
    if rows:
        df = pd.DataFrame(rows)
        display(df[['source_dir','path','width','height','bands','dtypes','crs','resolution']])
except Exception:
    pass

>> Revisando INTA-MNC:

------------------------------------------------------------
Archivo: /Users/joanespada/Desktop/ADE/Recorte&Visualizacion/ade_modelo_agricola/INTA-MNC/MNC_invierno2023.tif
  width x height: 43994 x 70641
  bands: 1
  dtype(s): ('uint8',)
  crs: EPSG:4326
  resolution: (0.0002694945852388963, 0.0002694945852408658)
  bounds: (-67.476323746, -41.035940494, -55.620178963, -21.998573498)

------------------------------------------------------------
Archivo: /Users/joanespada/Desktop/ADE/Recorte&Visualizacion/ade_modelo_agricola/INTA-MNC/MNC_verano2024.tif
  width x height: 43994 x 70641
  bands: 1
  dtype(s): ('uint8',)
  crs: EPSG:4326
  resolution: (0.0002694945852388963, 0.0002694945852408658)
  bounds: (-67.476323746, -41.035940494, -55.620178963, -21.998573498)

------------------------------------------------------------
Archivo: /Users/joanespada/Desktop/ADE/Recorte&Visualizacion/ade_modelo_agricola/INTA-MNC/recorte_invierno_GTiff.tif
  width x height: 4979 x

,source_dir,path,width,height,bands,dtypes,crs,resolution
0,INTA-MNC,/Users/joanespada/Desktop/ADE/Recorte&Visualiz...,43994,70641,1,"(uint8,)",EPSG:4326,"(0.0002694945852388963, 0.0002694945852408658)"
1,INTA-MNC,/Users/joanespada/Desktop/ADE/Recorte&Visualiz...,43994,70641,1,"(uint8,)",EPSG:4326,"(0.0002694945852388963, 0.0002694945852408658)"
2,INTA-MNC,/Users/joanespada/Desktop/ADE/Recorte&Visualiz...,4979,4998,1,"(float32,)",EPSG:32721,"(10.0, 10.0)"
3,INTA-MNC,/Users/joanespada/Desktop/ADE/Recorte&Visualiz...,4979,4998,1,"(float32,)",EPSG:32721,"(10.0, 10.0)"
4,Rasters-NDVI,/Users/joanespada/Desktop/ADE/Recorte&Visualiz...,2890,2897,1,"(float32,)",EPSG:32721,"(10.0, 10.0)"
5,Rasters-NDVI,/Users/joanespada/Desktop/ADE/Recorte&Visualiz...,2890,2897,1,"(float32,)",EPSG:32721,"(10.0, 10.0)"
6,Rasters-NDVI,/Users/joanespada/Desktop/ADE/Recorte&Visualiz...,2890,2897,1,"(float32,)",EPSG:32721,"(10.0, 10.0)"
7,Rasters-NDVI,/Users/joanespada/Desktop/ADE/Recorte&Visualiz...,2890,2897,1,"(float32,)",EPSG:32721,"(10.0, 10.0)"
8,Rasters-NDVI,/Users/joanespada/Desktop/ADE/Recorte&Visualiz...,2890,2897,1,"(float32,)",EPSG:32721,"(10.0, 10.0)"
9,Rasters-NDVI,/Users/joanespada/Desktop/ADE/Recorte&Visualiz...,2890,2897,1,"(float32,)",EPSG:32721,"(10.0, 10.0)"


In [14]:
# Recortar INTA-MNC con la resolución y bounds EXACTOS de NDVI
import subprocess
import json
from pathlib import Path

mnc_dir = Path(r"/Users/joanespada/Desktop/ADE/Recorte&Visualizacion/ade_modelo_agricola/INTA-MNC")
out_dir = Path(r"/Users/joanespada/Desktop/ADE/Recorte&Visualizacion/ade_modelo_agricola/INTA-MNC-recortado")

# Crear la carpeta de salida
print(f"📁 Creando carpeta de salida: {out_dir}")
out_dir.mkdir(parents=True, exist_ok=True)
print(f"✓ Carpeta creada/verificada\n")

# Obtener georeferencia exacta de NDVI
ndvi_dir = Path(r"/Users/joanespada/Desktop/ADE/Recorte&Visualizacion/ade_modelo_agricola/Rasters-NDVI")

# Buscar primer archivo NDVI que exista
ndvi_files = sorted(ndvi_dir.glob("NDVI_*.tif"))
if not ndvi_files:
    print(f"❌ Error: No se encontraron archivos NDVI_*.tif en {ndvi_dir}")
    print(f"Archivos disponibles: {list(ndvi_dir.glob('*.tif'))}")
else:
    ndvi_ref = ndvi_files[0]
    print(f"📊 Leyendo metadatos de referencia: {ndvi_ref.name}")
    
    info_json = subprocess.check_output(['gdalinfo', '-json', str(ndvi_ref)])
    ndvi_meta = json.loads(info_json)
    
    # Extraer información
    corners = ndvi_meta.get('cornerCoordinates', {})
    ulx = corners.get('upperLeft', [0, 0])[0]
    uly = corners.get('upperLeft', [0, 0])[1]
    lrx = corners.get('lowerRight', [0, 0])[0]
    lry = corners.get('lowerRight', [0, 0])[1]
    
    geotransform = ndvi_meta.get('geoTransform', [])
    pixel_width = geotransform[1] if len(geotransform) > 1 else 0
    pixel_height = geotransform[5] if len(geotransform) > 5 else 0
    
    print(f"   GeoTransform: {geotransform}")
    print(f"   Resolución: {pixel_width} x {pixel_height}")
    print(f"   Bounds: ({ulx}, {lry}) a ({lrx}, {uly})\n")
    
    # Archivos a recortar
    files = ["recorte_invierno_GTiff.tif", "recorte_verano_GTiff.tif"]
    
    for filename in files:
        src = mnc_dir / filename
        dst = out_dir / filename
        
        if not src.exists():
            print(f"❌ {filename} - Archivo no encontrado en {mnc_dir}")
            continue
        
        print(f"🔄 Procesando: {filename}")
        
        # gdalwarp con bounds y resolución exacta de NDVI
        cmd = [
            'gdalwarp',
            '-overwrite',
            '-of', 'GTiff',
            '-te', str(ulx), str(lry), str(lrx), str(uly),
            '-tr', str(pixel_width), str(abs(pixel_height)),
            str(src),
            str(dst)
        ]
        
        subprocess.run(cmd, check=True)
        
        # Verificar resultado
        result_info = subprocess.check_output(['gdalinfo', str(dst)])
        info_lines = result_info.decode().split('\n')
        # Buscar línea con tamaño
        for line in info_lines[:20]:
            if 'Size is' in line:
                print(f"   ✓ {line.strip()}")
                break
        print(f"   ✓ Guardado en: {dst}\n")
    
    print(f"✅ Proceso completado. Archivos en: {out_dir}")


📁 Creando carpeta de salida: /Users/joanespada/Desktop/ADE/Recorte&Visualizacion/ade_modelo_agricola/INTA-MNC-recortado
✓ Carpeta creada/verificada

📊 Leyendo metadatos de referencia: NDVI_2023-06.tif
   GeoTransform: [199060.0, 10.0, 0.0, 5762170.0, 0.0, -10.0]
   Resolución: 10.0 x -10.0
   Bounds: (199060.0, 5733200.0) a (227960.0, 5762170.0)

🔄 Procesando: recorte_invierno_GTiff.tif
Creating output file that is 2890P x 2897L.
Using internal nodata values (e.g. -9999) for image /Users/joanespada/Desktop/ADE/Recorte&Visualizacion/ade_modelo_agricola/INTA-MNC/recorte_invierno_GTiff.tif.
Copying nodata values from source /Users/joanespada/Desktop/ADE/Recorte&Visualizacion/ade_modelo_agricola/INTA-MNC/recorte_invierno_GTiff.tif to destination /Users/joanespada/Desktop/ADE/Recorte&Visualizacion/ade_modelo_agricola/INTA-MNC-recortado/recorte_invierno_GTiff.tif.
Processing recorte_invierno_GTiff.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
   GeoTransform: [1990